In [6]:
from pprint import pprint
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import math
# import os

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None

if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../../scratch/credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = discovery.build('sheets', 'v4', credentials=creds)

# The ID of the spreadsheet to update.
spreadsheet_id = '1jxr1XXez8EoNq-fbs0oXzBAR_39V0iaRZVifcKFqcoQ'

# The A1 notation of a range to search for a logical table of data.
# Values will be appended after the last row of the table.
range_ = 'england_table!A2'

# How the input data should be interpreted.
value_input_option = 'RAW'

# How the input data should be inserted.
insert_data_option = 'OVERWRITE'

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=627764670661-ckkcglb7qsmj4hjej6bvbr628osl029k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51072%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=t1cRBKeui28iWd0rUmPtpVhRiXgY10&access_type=offline


KeyboardInterrupt: 

In [ ]:
def fte_mvp(df):
    home = df.groupby('team1').apply(
        lambda x: (x['score1'] > x['score2']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'],
                        index = 'team1', aggfunc=sum)
    home = home.merge(pt.reset_index(), on = 'team1')
    pt = pd.pivot_table(df, values = 'date', index = 'team1', aggfunc=pd.Series.nunique)
    home = home.merge(pt.reset_index(), on = 'team1')
    cols = {'date':'played', 'team1':'team', 'score1':'scored', 'score2':'conceded', 'xg1':'xg_scored',
            'xg2':'xg_conceded', 'nsxg1':'nsxg_scored', 'nsxg2':'nsxg_conceded', 'adj_score1':'adj_goals_scored',
            'adj_score2':'adj_goals_conceded'}
    home = home.rename(columns = cols)
    away = df.groupby('team2').apply(
        lambda x: (x['score2'] > x['score1']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'],
                        index = 'team2', aggfunc=sum)
    away = away.merge(pt.reset_index(), on = 'team2')
    pt = pd.pivot_table(df, values = 'date', index = 'team2', aggfunc=pd.Series.nunique)
    away = away.merge(pt.reset_index(), on = 'team2')
    cols = {'date':'played', 'team2':'team', 'score2':'scored', 'score1':'conceded', 'xg2':'xg_scored',
            'xg1':'xg_conceded', 'nsxg2':'nsxg_scored', 'nsxg1':'nsxg_conceded', 'adj_score2':'adj_goals_scored',
            'adj_score1':'adj_goals_conceded'}
    away = away.rename(columns = cols)
    t = pd.concat([home, away], ignore_index=True)
    t = t.groupby('team').sum()
    t['goal_difference'] = t['scored'] - t['conceded']
    t['xg_difference'] = t['xg_scored'] - t['xg_conceded']
    t['nsxg_difference'] = t['nsxg_scored'] - t['nsxg_conceded']
    t['adj_goal_difference'] = t['adj_goals_scored'] - t['adj_goals_conceded']
    return t

In [ ]:
def fte_england_tb(df):
    t = fte_mvp(df)
    t = t.sort_values(by=['points', 'goal_difference', 'scored'], ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [35]:
def fte_export_gs(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        tb = getattr('_'.join(['fte', key, 'tb']))
        num_teams = len(league.loc[:50, 'team1'].unique())
        for i in range(math.ceil(league.shape[0]/(num_teams * (num_teams - 1)))):
            temp = league.loc[((num_teams * (num_teams - 1)) * i):((num_teams * (num_teams - 1)) * (i+1) - 1),:]
            season = temp.iloc[0]['date'][0:4]
            season = '-'.join([season,str(int(season) + 1)])
            t = fte_mvp(temp)
            t = tb(t)
            t = t.insert(0,'season',season)
            
    return t

In [36]:
path = os.path.expanduser('~/Downloads/spi_matches.csv')
league_dict = {'england':'2411'}
# league_dict = {'spain':'1869'}
#, 'italy':'1854',
# league_dict = {'germany':'1845'} #, 'france':'1843'}
df = fte_export_gs(path, league_dict)

TypeError: getattr expected at least 2 arguments, got 1

In [32]:
df

,season,index,date,league_id,league,team1,team2,spi1,spi2,prob1,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
918,2019-2020,23396,2019-08-16,1845,German Bundesliga,Bayern Munich,Hertha Berlin,93.98,66.45,0.8893,...,66.0,26.0,2.0,2.0,2.61,0.76,3.63,0.57,2.10,2.10
919,2019-2020,23424,2019-08-17,1845,German Bundesliga,SC Freiburg,Mainz,60.96,67.32,0.3941,...,33.4,22.1,3.0,0.0,1.83,0.96,1.23,1.45,2.38,0.00
920,2019-2020,23426,2019-08-17,1845,German Bundesliga,Borussia Dortmund,FC Augsburg,83.53,59.46,0.7959,...,60.8,26.6,5.0,1.0,3.05,0.70,2.18,0.46,4.95,1.05
921,2019-2020,23427,2019-08-17,1845,German Bundesliga,Bayer Leverkusen,SC Paderborn,82.59,58.32,0.7970,...,60.2,37.4,3.0,2.0,2.27,1.27,1.15,0.87,3.15,2.10
922,2019-2020,23428,2019-08-17,1845,German Bundesliga,VfL Wolfsburg,FC Cologne,70.78,62.36,0.5561,...,31.0,27.2,2.0,1.0,0.58,0.96,1.16,0.86,2.10,1.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,2019-2020,33408,2020-06-27,1845,German Bundesliga,Werder Bremen,FC Cologne,60.66,71.12,0.3294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1220,2019-2020,33409,2020-06-27,1845,German Bundesliga,Bayer Leverkusen,Mainz,81.40,60.30,0.7296,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221,2019-2020,33410,2020-06-27,1845,German Bundesliga,1. FC Union Berlin,Fortuna Düsseldorf,62.34,58.11,0.4813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1222,2019-2020,33411,2020-06-27,1845,German Bundesliga,FC Augsburg,RB Leipzig,63.05,87.91,0.1741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
